# Introduction to multivariable Green functions

In this notebook, we will learn how to manipulate multivariable Green functions. As
an example, we will consider the Green's function on a square lattice with nearest-neighbour hopping $t$,

\begin{equation}
G_0(\mathbf{k},i\omega_n)=\frac{1}{i\omega_n  + \mu - \epsilon(\mathbf{k})}
\end{equation}

whose dispersion is $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})$, where $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$), $\mu$ is the chemical potential and $i\omega_n$ is a Matsubara frequency.

## Imports and parameters

Below we import modules that will be useful in the following. We also set the
parameters of the problem.

In [ ]:
# Imports 
%matplotlib inline
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import Gf, MeshProduct, MeshBrillouinZone, MeshImFreq
from pytriqs.plot.mpl_interface import plt
import numpy as np
import warnings 
warnings.filterwarnings("ignore") #ignore some matplotlib warnings
from math import cos, pi
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figures

# Regroup some parameters of the computation used later
beta = 1/0.4 # Inverse temperature
t = 1.0      # Hopping   
n_k = 128    # Number of points in the Brillouin Zone mesh (for each dimension)
n_w = 128    # Number of Matsubara frequencies
mu = 0       # Chemical potential

## A mesh on a Brillouin Zone

We first define a simple Bravais lattice (`BravaisLattice`) in 2 dimensions with basis vectors $\hat{e}_x = (1, 0, 0)$ and ${\hat e}_y=(0, 1, 0)$, and given the bravais lattice we construct the reciprocal (momentum) space Brillouin zone (`BrillouinZone`).

In [ ]:
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)]) # Two unit vectors in R3
BZ = BrillouinZone(BL) 

The complex valued Green's function $G(\mathbf{k}, i\omega_n) $ is represented on the cartesian product mesh : $ (\mathbf{k} \times i\omega_n) \rightarrow {\mathcal{C}}$. 

We construct $G$ by first defining the two separate meshes in momentum space $\mathbf{k}$ (`MeshBrillouinZone`) and frequency space $i\omega_n$ (`MeshImFreq`) and then use the `MeshProduct` of these meshes as the mesh for $G(\mathbf{k}, i\omega_n)$. Note that the `target_shape=[]` parameter means that, for a given $\mathbf{k}$ and a given $i\omega_n$, $G(\mathbf{k}, i\omega_n)$ will be a scalar (instead of a $2 \times 2$ matrix for instance).

In [ ]:
kmesh = MeshBrillouinZone(BZ, n_k=n_k)
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g0 = Gf(mesh=MeshProduct(kmesh, wmesh), target_shape=[])  # g0(k,omega), scalar valued

To fill the Green's function we construct a function for the dispersion $\epsilon(\mathbf{k})$ and set each element of $G$ by looping over the momentum and frequency meshes.

In [ ]:
#%%timeit
def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g0.mesh[0]:
    for w in g0.mesh[1]:
        g0[k, w] = 1/(w - eps(k))

## (Optional) More advanced

Initializing the Green function with the cell above is quite slow. The reason is that loops are slow in Python. It is a good opportunity to illustrate a bit the C++ layer of the TRIQS library, using the TRIQS/cpp2py tool to wrap Python and C++ in a simple case.

We first import an ipython "magic" command `%%cpp2py` :

In [ ]:
%reload_ext cpp2py.magic

The function `compute_g0` below is written in C++. 
It takes a Green's function, and the hopping $t$
and does the same computation as above.

When executing the "magic" cell, the C++ is analysed by the Clang compiler, 
a wrapping code (to expose it to Python and convert its argument) is generated,
compiled. The whole is linked into a module, which is loaded automatically,
so the `compute_g0` function is available in Python.


In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

void compute_g0(gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued> g, double t) {
      for (auto [k,w] : g.mesh()) // loop on points
       g[k,w] = 1/(w - (-2)*t *(cos(k[0]) + cos(k[1]))); 
       // k,w are points on the grid and cast in points on the k mesh, and a Matsubara frequency resp.
}

In [ ]:
#%%timeit
# we call the function and check that we get the same answer...
g0b = Gf(mesh=MeshProduct(kmesh, wmesh), target_shape=[])  # g0(k,omega), scalar valued
compute_g0(g0b,t)

assert np.amax(np.abs(g0b.data - g0.data)) < 1.e-14, "It should be the same ..."

NB : If you uncomment the `%%timeit` command in the previous cells, python will time the two functions and you will be able to see the difference of speed.    

## Evaluate the Green function

The Green function object $g_0(k,i\omega_n)$ can be evaluated like an ordinary Python function
at a given reciprocal vector and Matsubara frequency:

- The reciprocal vector $k$ is a tuple/list/numpy.array of double 
- The Matsubara frequency is an integer $n$, the $n$ in $i\omega_n$

The result will be a linear interpolation on the Brillouin zone 
  with the points on the grid of $g_0$ around $k$.

Therefore, one can use $g_0$ as any python function of $k$ and $i\omega_n$, 
and forget its precise representation in memory (what is the grid, etc...).
We will use that in the plot functions below.

Example:
Let's evaluate the above Green's function at $\mathbf{k} = (\pi,\pi,0)$ and $i\omega_2$. As $\epsilon((\pi,\pi,0)) = 2t = 2$ and $i\omega_2 = i\frac{(2*2 + 1)\pi}{\beta}$, we check:

In [ ]:
print g0((pi,pi,0), 2) -  1/( 1j * (2*2 +1) *pi/beta - 4) # little check

## Partial evaluation

Given a function $g(k,i\omega_n)$, for a given $k_0$, it is possible to obtain
the function $i\omega_n \rightarrow g(k_0, i\omega_n)$ :


In [ ]:
k0 = (0.02,0.01,0)      # a k point as a tuple of 3 floats
gw = g0(k0, all)        # We use the "built-in" function all here as equivalent of :, 
                        # which Python does not allow in ()
                        # gw is now a function of $\omega$ only, interpolated in k
            
assert gw.mesh == g0.mesh[1]            # The meshes in indeed a simple ImFreq mesh now
assert abs(gw(0) - g0(k0, 0)) < 1.e-14  # Partially evaluate the function and use it, or directly evaluate it, is the same

The new function is interpolated linearly for the point $k_0$ on the original Brillouin zone grid. The interpolated Green function is now nothing else but the usual Matsubara
frequency Green function that you have used earlier. This means you can use all the
methods for this Green function, such as `density()` or Fourier transforms.
Example:

In [ ]:
# This is the density n_k at k=(0.02, 0.01)
print "n_k =",  gw.density().real

Note here that, compared to what you may have done in the first tutorials, you don't need to take the `[0,0]` component of `gw.density()` because you specified in the definition of the Green's function that it should have scalar values, `target_shape=[]`.